In [241]:
with open("names.txt") as f:
    names = f.read().splitlines()

# if I have a very rigid dataset
# names = ["lorenc", "lore", "lorec"] * 1000
names = ["lorenc"] * 3000

In [242]:
len(names)

3000

In [243]:
chars = ["#"] + list(set("".join(names).lower()))
len(chars)

7

In [244]:
from collections import defaultdict
bigram_counts = defaultdict(dict)
for n in names:
    tokens = ["#"] + list(n.lower()) + ["#"]
    for first_token, second_token in zip(tokens[:-1], tokens[1:]):
         bigram_counts[first_token][second_token] = bigram_counts[first_token].get(second_token, 0) + 1

In [245]:
list(sorted(bigram_counts["#"].items(), key=lambda x: -x[1]))

[('l', 3000)]

In [246]:
import torch
N = torch.zeros((len(chars), len(chars)), dtype=torch.int32)
N = N + 1  # add one smoothing

In [247]:
chars[:5]

['#', 'l', 'o', 'e', 'c']

In [248]:
char2idx = {c: i for i, c in enumerate(sorted(chars))}
idx2char = {i: c for i, c in enumerate(sorted(chars))}
for first_token, second_tokens in bigram_counts.items():
    for second_token, count in second_tokens.items():
        N[char2idx[first_token], char2idx[second_token]] = count
N

tensor([[   1,    1,    1, 3000,    1,    1,    1],
        [3000,    1,    1,    1,    1,    1,    1],
        [   1,    1,    1,    1, 3000,    1,    1],
        [   1,    1,    1,    1,    1, 3000,    1],
        [   1, 3000,    1,    1,    1,    1,    1],
        [   1,    1,    1,    1,    1,    1, 3000],
        [   1,    1, 3000,    1,    1,    1,    1]], dtype=torch.int32)

In [249]:
N[char2idx["#"], char2idx["l"]]

tensor(3000, dtype=torch.int32)

In [250]:
g = torch.Generator().manual_seed(42)
selected = torch.multinomial(N[char2idx["#"]].float(), num_samples=1, replacement=True, generator=g).item()
idx2char[selected]

'l'

In [251]:
g = torch.Generator().manual_seed(42)

for _ in range(5):
    output = []
    selected = char2idx["#"]
    while True:
        selected = torch.multinomial(N[selected].float(), num_samples=1, replacement=True, generator=g).item()
        if selected == char2idx["#"]:
            break
        output.append(idx2char[selected])
    print("".join(output))


lorenc
lorenc
lorenc
lorenc
lorenc


In [252]:
# https://jiselectric.github.io/posts/NLL/

# Probability refers to the chance that a particular outcome occurs based on the values of parameters in a model.
# Likelihood refers to how well a sample provides support for particular values of a parameter in a model.

# Suppose we have a coin that is assumed to be fair. If we flip the coin one time, the probability that it will land on heads is 0.5.

# Now suppose we flip the coin 100 times and it only lands on heads 17 times. We would say that the likelihood that the coin is fair is quite low. If the coin was actually fair, we would expect it to land on heads much more often.

In [253]:
# likelihood
# names_to_evaluate = ["lorenc"]
names_to_evaluate = names[:10]
for name in names_to_evaluate:
    tokens = ["#"] + list(name.lower()) + ["#"]
    log_likelihood = 0.0
    n = 0
    for first_token, second_token in zip(tokens, tokens[1:]):
        first_idx = char2idx[first_token]
        second_idx = char2idx[second_token]

        count = N[first_idx, second_idx].item()
        total_count = N[first_idx].sum().item()

        prob = count / total_count

        log_likelihood += torch.log(torch.tensor(prob))
        n += 1
print(f"{log_likelihood=}")
print(f"{-log_likelihood=}")
print(f"{-log_likelihood/n=}")

log_likelihood=tensor(-0.0140)
-log_likelihood=tensor(0.0140)
-log_likelihood/n=tensor(0.0020)


# Using Pytorch

In [128]:
# using Pytorch

In [146]:
names_for_training = names[:1]
context = []
target = []
for name in names_for_training:
    tokens = ["#"] + list(name.lower()) + ["#"]
    for first_token, second_token in zip(tokens, tokens[1:]):
        first_idx = char2idx[first_token]
        second_idx = char2idx[second_token]
        context.append(first_idx)
        target.append(second_idx)
print(f"{context=}")
print(f"{target=}")

context=[0, 1, 2, 17, 1, 8, 1, 13]
target=[1, 2, 17, 1, 8, 1, 13, 0]


In [147]:
print(f"{[idx2char[x] for x in context]=}")
print(f"{[idx2char[x] for x in target]=}")

[idx2char[x] for x in context]=['#', 'a', 'b', 'r', 'a', 'h', 'a', 'm']
[idx2char[x] for x in target]=['a', 'b', 'r', 'a', 'h', 'a', 'm', '#']


In [149]:
context = torch.tensor(context, dtype=torch.int64)
target = torch.tensor(target, dtype=torch.int64)

In [154]:
import torch.nn.functional as F
inputs = F.one_hot(torch.tensor([1,1,1,2,3,4,0]), 5)
inputs

tensor([[0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0]])

In [155]:
# W = torch.randn((len(chars), len(chars)), dtype=torch.float32, requires_grad=True)
W = torch.randn((5, 5), dtype=torch.float32, requires_grad=True)

In [157]:
inputs.float() @ W.float()

tensor([[ 0.8782,  0.5552,  0.6358,  1.4098,  0.1236],
        [ 0.8782,  0.5552,  0.6358,  1.4098,  0.1236],
        [ 0.8782,  0.5552,  0.6358,  1.4098,  0.1236],
        [ 0.4022, -0.1551,  1.3986,  0.2576,  1.4541],
        [ 0.2580,  0.7843,  0.2289,  1.3777, -0.0184],
        [ 0.8332,  1.0087,  0.3525,  0.8193, -0.1315],
        [-0.3564,  1.2193,  0.4721, -0.4148, -1.9721]], grad_fn=<MmBackward0>)

In [176]:
W[1,:] # first row, one hot encoding will just pick up this row
# we need to apply softmax - https://en.wikipedia.org/wiki/Softmax_function
logits = inputs.float() @ W.float()
logits = logits.exp()
print(logits.shape)
count = logits.sum(dim=1, keepdim=True)
print(count.shape)
# print(count)
probs =logits / count
print(probs)

torch.Size([7, 5])
torch.Size([7, 1])
tensor([[0.2136, 0.1547, 0.1677, 0.3636, 0.1005],
        [0.2136, 0.1547, 0.1677, 0.3636, 0.1005],
        [0.2136, 0.1547, 0.1677, 0.3636, 0.1005],
        [0.1249, 0.0715, 0.3382, 0.1080, 0.3575],
        [0.1336, 0.2261, 0.1297, 0.4093, 0.1013],
        [0.2394, 0.2853, 0.1480, 0.2361, 0.0912],
        [0.1079, 0.5217, 0.2471, 0.1018, 0.0214]], grad_fn=<DivBackward0>)


In [178]:
target

tensor([ 1,  2, 17,  1,  8,  1, 13,  0])

In [182]:
torch.arange(len(target)), target

(tensor([0, 1, 2, 3, 4, 5, 6, 7]), tensor([ 1,  2, 17,  1,  8,  1, 13,  0]))

In [185]:
probs[torch.arange(len([1,1,2,3,4,0])), [1,1,2,3,4,0]]

tensor([0.1547, 0.1547, 0.1677, 0.1080, 0.1013, 0.2394],
       grad_fn=<IndexBackward0>)

In [186]:
probs[[1,2,3], [1,2,3]]

tensor([0.1547, 0.1677, 0.1080], grad_fn=<IndexBackward0>)

# Full examples

In [254]:
# names_for_training = names[:10]
names_for_training = names
context = []
target = []
for name in names_for_training:
    tokens = ["#"] + list(name.lower()) + ["#"]
    for first_token, second_token in zip(tokens, tokens[1:]):
        first_idx = char2idx[first_token]
        second_idx = char2idx[second_token]
        context.append(first_idx)
        target.append(second_idx)
print(f"{context[:10]=}")
print(f"{target[:10]=}")

context[:10]=[0, 3, 5, 6, 2, 4, 1, 0, 3, 5]
target[:10]=[3, 5, 6, 2, 4, 1, 0, 3, 5, 6]


In [255]:
len(context)

21000

In [256]:
len(target)

21000

In [257]:
context = torch.tensor(context, dtype=torch.int64)
target = torch.tensor(target, dtype=torch.int64)
W = torch.randn((len(char2idx), len(char2idx)), dtype=torch.float32, requires_grad=True)

context = F.one_hot(context, num_classes=len(char2idx))

In [258]:
for epoch in range(1_000):
    logits = context.float() @ W.float() # pick up the line, W should represent probabilities
    logits = logits.exp()
    count = logits.sum(dim=1, keepdim=True)
    probs = logits / count
    loss = -torch.log(probs[torch.arange(len(context)), target]).mean()
    print(f"{loss.item()=}")
    W.grad = None  # reset gradients
    loss.backward()
    W.data -= 1 * W.grad


loss.item()=2.9746012687683105
loss.item()=2.8236284255981445
loss.item()=2.6773669719696045
loss.item()=2.5359787940979004
loss.item()=2.3996007442474365
loss.item()=2.268334150314331
loss.item()=2.142249822616577
loss.item()=2.0213849544525146
loss.item()=1.9057458639144897
loss.item()=1.7953016757965088
loss.item()=1.6899974346160889
loss.item()=1.5897561311721802
loss.item()=1.494479775428772
loss.item()=1.4040570259094238
loss.item()=1.3183672428131104
loss.item()=1.2372866868972778
loss.item()=1.1606863737106323
loss.item()=1.0884383916854858
loss.item()=1.0204212665557861
loss.item()=0.9565091133117676
loss.item()=0.8965794444084167
loss.item()=0.8405036926269531
loss.item()=0.7881523370742798
loss.item()=0.7393856048583984
loss.item()=0.6940559148788452
loss.item()=0.652006983757019
loss.item()=0.6130728125572205
loss.item()=0.5770790576934814
loss.item()=0.54384845495224
loss.item()=0.5131967067718506
loss.item()=0.4849432706832886
loss.item()=0.45890745520591736
loss.item()=0

In [259]:
g = torch.Generator().manual_seed(42)

for _ in range(5):
    output = []
    selected = char2idx["#"]
    while True:
        one_hot = F.one_hot(torch.tensor(selected), num_classes=len(char2idx)).float()
        logits = one_hot @ W.float()  # pick up the line, W should represent probabilities
        logits = logits.exp()
        count = logits.sum()
        probs = logits / count

        selected = torch.multinomial(probs, num_samples=1, replacement=True, generator=g).item()
        if selected == char2idx["#"]:
            break
        output.append(idx2char[selected])
    print("".join(output))


lorenc
lorenc
lorenc
lorenc
lorenc


In [260]:
logits.sum()

tensor(243.6905, grad_fn=<SumBackward0>)

In [268]:
# likelihood
# names_to_evaluate = ["lorenc"]
names_to_evaluate = names[:10]
with torch.no_grad():
    for name in names_to_evaluate:
        tokens = ["#"] + list(name.lower()) + ["#"]
        log_likelihood = 0.0
        n = 0
        for first_token, second_token in zip(tokens, tokens[1:]):
            first_idx = char2idx[first_token]
            second_idx = char2idx[second_token]

            one_hot = F.one_hot(torch.tensor([first_idx]), num_classes=len(char2idx)).float()
            logits = one_hot @ W.float()  # pick up the line, W should represent probabilities
            logits = logits.exp()
            count = logits.sum()
            probs = logits[0, second_idx] / count

            log_likelihood += torch.log(probs).mean().item()
            n += 1
    print(f"{log_likelihood=}")
    print(f"{-log_likelihood=}")
    print(f"{-log_likelihood/n=}")

log_likelihood=-0.04291695822030306
-log_likelihood=0.04291695822030306
-log_likelihood/n=0.006130994031471866


In [269]:
logits

tensor([[2.4223e+02, 9.6842e-02, 2.7559e-01, 3.1765e-01, 3.1503e-01, 1.0683e-01,
         3.4852e-01]])

In [265]:
second_idx

3